In [ ]:
!git clone https://github.com/AvonYangXX1/AMPLify-Feedback.git
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

Cloning into 'AMPLify-Feedback'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 106 (delta 66), reused 98 (delta 61), pack-reused 0
Receiving objects: 100% (106/106), 6.40 MiB | 4.28 MiB/s, done.
Resolving deltas: 100% (66/66), done.
Updating files: 100% (41/41), done.


In [ ]:
# Load data
seq_train = np.load("AMPLify-Feedback/processed_data/cv/seq_train_0.npy")
state_train = np.load("AMPLify-Feedback/processed_data/cv/state_train_0.npy")
label_train = np.load("AMPLify-Feedback/processed_data/cv/label_train_0.npy")
seq_test = np.load("AMPLify-Feedback/processed_data/cv/seq_test_0.npy")
state_test = np.load("AMPLify-Feedback/processed_data/cv/state_test_0.npy")
label_test = np.load("AMPLify-Feedback/processed_data/cv/label_test_0.npy")
seq_train = np.expand_dims(seq_train, 2)
seq_test = np.expand_dims(seq_test, 2)

# Convert to torch tensors
seq_train_tensor = torch.from_numpy(seq_train).float()
state_train_tensor = torch.from_numpy(state_train).float()
label_train_tensor = torch.from_numpy(label_train).float()
seq_test_tensor = torch.from_numpy(seq_test).float()
state_test_tensor = torch.from_numpy(state_test).float()
label_test_tensor = torch.from_numpy(label_test).float()

# Create dataloaders
train_dataset = TensorDataset(seq_train_tensor, state_train_tensor, label_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = TensorDataset(seq_test_tensor, state_test_tensor, label_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
class ActivityPredictor(nn.Module):
    def __init__(self):
        super(ActivityPredictor, self).__init__()
        # RNN
        self.gru_0 = nn.LSTM(1, 1024, batch_first=True)
        # self.gru_1 = nn.LSTM(512, 512)
        # self.gru_2 = nn.GRU(512, 512)
        self.gru_3 = nn.LSTM(1024, 128)
        self.linear_rnn_0 = nn.Linear(128, 128)
        # self.layernorm_rnn = nn.LayerNorm(128)
        # Embedding
        self.linear_embed = nn.Linear(326, 128)
        self.layernorm_embed = nn.LayerNorm(128)
        # FCN
        self.linear_fcn_0 = nn.Linear(256, 1024)
        self.linear_fcn_1 = nn.Linear(1024, 1024)
        self.linear_fcn_2 = nn.Linear(1024, 64)
        self.layernorm_fcn = nn.LayerNorm(64)
        # forward
        self.layernorm_fwd = nn.LayerNorm(256)
        # output
        self.linear_output = nn.Linear(64, 1)

    def forward(self, seq, state):
        x0 = self.rnn(seq)
        x1 = self.embed(state)
        x = torch.cat((x0, x1), dim=1)
        x = self.layernorm_fwd(x)
        x = nn.ReLU()(x)
        x = self.fcn(x)
        x = self.output(x)
        return x

    def embed(self, x):
        x = self.linear_embed(x)
        x = nn.ReLU()(x)
        x = self.layernorm_embed(x)
        x = nn.ReLU()(x)
        return x

    def rnn(self, x):
        x, _ = self.gru_0(x)
        # x, _ = self.gru_1(x)
        # x, _ = self.gru_2(x)
        x, _ = self.gru_3(x)
        x = x[:, -1, :]
        x = self.linear_rnn_0(x)
        x = nn.ReLU()(x)
        # x = self.layernorm_rnn(x)
        return x

    def fcn(self, x):
        x = self.linear_fcn_0(x)
        x = nn.ReLU()(x)
        x = self.linear_fcn_1(x)
        x = nn.ReLU()(x)
        x = self.linear_fcn_2(x)
        x = nn.ReLU()(x)
        x = self.layernorm_fcn(x)
        x = nn.ReLU()(x)
        return x

    def output(self, x):
        x = self.linear_output(x)
        # x = nn.Sigmoid()(x)
        return x



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ActivityPredictor().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.BCEWithLogitsLoss()

EPOCHS = 10

for epoch in range(EPOCHS):
    # Training Loop
    model.train()
    total_loss = 0
    correct_train = 0
    for batch_seqs, batch_states, batch_labels in train_loader:
        batch_seqs, batch_states, batch_labels = batch_seqs.to(device), batch_states.to(device), batch_labels.to(device)
        optimizer.zero_grad()
        outputs = model(seq=batch_seqs, state=batch_states)
        loss = criterion(outputs, batch_labels.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct_train += ((outputs.squeeze() > 0.5) == batch_labels).sum().item()

    train_accuracy = correct_train / len(seq_train)

    # Validation Loop
    model.eval()
    total_val_loss = 0
    correct_val = 0
    with torch.no_grad():
        for batch_seqs, batch_states, batch_labels in test_loader:
            batch_seqs, batch_states, batch_labels = batch_seqs.to(device), batch_states.to(device), batch_labels.to(device)
            outputs = model(batch_seqs, batch_states)
            loss = criterion(outputs, batch_labels.float())
            total_val_loss += loss.item()
            correct_val += ((outputs.squeeze() > 0.5) == batch_labels).sum().item()

    val_accuracy = correct_val / len(seq_test)

    # Logging Results
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    print('-' * 40)
    print(f"Train Loss: {total_loss / len(train_loader):.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Validation Loss: {total_val_loss / len(test_loader):.4f}, Validation Accuracy: {val_accuracy:.4f}")


Epoch 1/10
----------------------------------------
Train Loss: 0.6928, Train Accuracy: 15.9550
Validation Loss: 0.6918, Validation Accuracy: 15.9430
Epoch 2/10
----------------------------------------
Train Loss: 0.6913, Train Accuracy: 15.9966
Validation Loss: 0.6912, Validation Accuracy: 15.9430
Epoch 3/10
----------------------------------------
Train Loss: 0.6909, Train Accuracy: 15.9970
Validation Loss: 0.6918, Validation Accuracy: 15.9430
Epoch 4/10
----------------------------------------
Train Loss: 0.6906, Train Accuracy: 15.9948
Validation Loss: 0.6920, Validation Accuracy: 15.9430


KeyboardInterrupt: ignored

In [ ]:
nn.Sigmoid()(model(seq=batch_seqs, state=batch_states))

tensor([[0.5176],
        [0.4882],
        [0.4895],
        [0.4895],
        [0.4962],
        [0.4896],
        [0.5176],
        [0.4769],
        [0.4774],
        [0.4925],
        [0.4929],
        [0.4979],
        [0.4979],
        [0.4799],
        [0.4896],
        [0.4916],
        [0.4799],
        [0.4896],
        [0.4850],
        [0.4769],
        [0.4896],
        [0.4799],
        [0.4799],
        [0.4850],
        [0.4836],
        [0.4775],
        [0.4506],
        [0.4881],
        [0.4979],
        [0.4895],
        [0.4896],
        [0.4772]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [ ]:
batch_labels

tensor([[0.0000],
        [0.0000],
        [0.9971],
        [0.0000],
        [0.9430],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.9971],
        [0.9942],
        [0.0000],
        [0.9968],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.9932],
        [0.9873],
        [0.9845],
        [0.9955],
        [0.9910],
        [0.7811],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.5458],
        [0.9965],
        [0.0000],
        [0.9973],
        [0.9968],
        [0.7699]], device='cuda:0')